# Target Trial Emulation (Data Analysis - Assignment 1: Clustering)
---

### Step 1: Install TrialEmulation package using R

In [ ]:
if (!requireNamespace("TrialEmulation", quietly = TRUE)) {
  install.packages("TrialEmulation")
}
library(TrialEmulation)

##### Next, load the package and export it to a .csv file.

In [ ]:
library(TrialEmulation)

data("data_censored")

write.csv(data_censored, "data_censored.csv", row.names = FALSE)

---

### Step 2: Obtain the R code from this link 'https://rpubs.com/alanyang0924/TTE'

##### Save it as a separate file, in our case it's 'TTE.r'.
---

### Step 3: Convert the R code into Python code